In [7]:
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import random as ra
import matplotlib.pyplot as plt
import sqlite3
import datetime
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score


In [8]:
with open('colors.txt') as file:
    colors = file.read().replace('\n', ' ').split(', ')
amount = 30
titles = ['Динамический диапазон', 'Угол обзора камеры', 'Фокусное расстояние', 'Температура', 'Частота колебаний']
data = pd.read_excel('database.xlsx')[:amount]
parameters = list(data.columns[1:])
transcripts = {column: title for column, title in zip(parameters, titles)}
scopes = {columns: ranges for columns, ranges in zip(parameters,
                                               [(4, 9), (6, 160), (2.8, 16), (-10, 60), (8, 120)])}
display(data)
scopes

,time,dynamic_range,viewing_angle,focal_length,temperature,oscillation_frequency
0,16.07.2009,8,76,12.130096,48,75
1,17.07.2009,7,133,9.612135,45,103
2,18.07.2009,7,63,11.103212,54,23
3,19.07.2009,6,129,10.750230,56,71
4,20.07.2009,8,122,14.644767,21,14
5,21.07.2009,8,20,11.495282,33,68
6,22.07.2009,4,110,9.305540,14,34
7,23.07.2009,4,39,9.514850,-10,12
8,24.07.2009,8,112,15.712198,-4,46
9,25.07.2009,8,141,10.707476,38,53


{'dynamic_range': (4, 9),
 'viewing_angle': (6, 160),
 'focal_length': (2.8, 16),
 'temperature': (-10, 60),
 'oscillation_frequency': (8, 120)}

In [2]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except BaseException as e:
        print(e)

    return conn


def select_all_readings(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM readings")

    rows = cur.fetchall()

    for row in rows:
        print(row)


database = r"main_database.db"

conn = create_connection(database)
# with conn:
#     select_all_readings(conn)

In [3]:
corr = data.corr()
display(corr.style.background_gradient(cmap='coolwarm'))
weights = (corr.apply(abs).reset_index(drop=True).sum() - 1) / 4
weights

,dynamic_range,viewing_angle,focal_length,temperature,oscillation_frequency
dynamic_range,1.000000,-0.028862,0.033434,0.329010,0.222667
viewing_angle,-0.028862,1.000000,0.205269,0.084173,0.131679
focal_length,0.033434,0.205269,1.000000,-0.069227,-0.208640
temperature,0.329010,0.084173,-0.069227,1.000000,0.164623
oscillation_frequency,0.222667,0.131679,-0.208640,0.164623,1.000000


dynamic_range            0.153493
viewing_angle            0.112496
focal_length             0.129142
temperature              0.161758
oscillation_frequency    0.181902
dtype: float64

In [4]:
def predict_ridge_model(model, size, y, scope):
    predicted_x, begin, end = [], scope[0], scope[1]
    size -= size % 5
    for i in range(0, size, 5):
        x = model.predict([y[i:i + 5]])[0]
        predicted_x.append(x)
        if x < begin or end < x:
            break
    else:
        return f'{size}+'
    return len(predicted_x)

def forming_characteristics(data, column, scope):
    data = data[parameters]
    X, y = data.values, data[column].values

    model = Ridge(alpha=1.0)

    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=100)

    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

    scores = np.absolute(scores)
    model.fit(X, y)
    predicted_x = predict_ridge_model(model, len(data), y, scope)
    print(f'{column}: {predicted_x}')
    print(f'Mean MAE: {np.mean(scores)} ({np.std(scores)})')
    return predicted_x

def general_predict(predict_days):
    summary = 0
    for column in parameters:
        weight, days = weights[column], predict_days[column]
        try:
            summary += weight * days
        except TypeError:
            days = int(days[:-1])
            summary += weight ** 2 * days
    return round(summary)


def predict_parameters(begin, end):
    for column in parameters:
        predict_days[column] = forming_characteristics(data[begin:end], column, scope=scopes[column])
    return predict_days


def form_days(day):
    if day == 1:
        return '1 дня'
    return f'{day} дней'

def rebuild_date(starting_date, days):
    return (starting_date + datetime.timedelta(days=days)).strftime("%d.%m.%Y")


def create_figure(data, begin, end):
    data = data[begin:end]
    fig = make_subplots(rows=5, cols=1, vertical_spacing=0.1, subplot_titles=titles)
    for column, i in zip(parameters, range(1, 6)):
        fig.add_trace(px.line(data, x=data.time, y=column)['data'][0], row=i, col=1)
        fig.update_traces(line_color=ra.choice(colors), row=i, col=1)
    fig.update_layout(autosize=False, width=1800, height=1200)
    return fig


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
begin, end, predict_days = 10, 200, {}
starting_date = datetime.datetime.strptime(data.time[begin], "%d.%m.%Y")
predict_days = predict_parameters(begin, end)
app.layout = html.Div([html.H2('FZ-SC2M 374930'),
                       dcc.Graph(id='main-graph', figure=create_figure(data, begin, end)),
                       html.Div(id='description', children='Введите номер параметра'),
                       html.Div(dcc.Input(id='input-on-submit', type='text')),
                       html.Button('Выполнить', id='submit-val', n_clicks=0),
                       html.Div(id='container-button')])


@app.callback(
    dash.Output('container-button', 'children'),
    dash.Input('submit-val', 'n_clicks'),
    dash.State('input-on-submit', 'value'))


def update_output(n_clicks, value):
    if not n_clicks:
        return None
    try:
        column = parameters[int(value) - 1]
        days = predict_days[column]
        if value == '0':
            days = general_predict(predict_days)
            return f'Проведите технический осмотр датчика в течение {form_days(days)} (до {rebuild_date(starting_date, days)})'
        if type(days) == str:
            days = int(days[:-1])
            date = rebuild_date(starting_date, days)
            return f'В течение {form_days(days)} (до {date}) параметр "{transcripts[column]}" будет находиться в пределах нормы'
        date = rebuild_date(starting_date, days)
        return f'Проверьте параметр "{transcripts[column]}" датчика в течение {form_days(days)} (до {date})'
    except TypeError:
        return f'Введите корректное значение'


if __name__ == '__main__':
    app.run_server(debug=False)


dynamic_range: 20+
Mean MAE: 0.05501079990023096 (0.06417955486507568)
viewing_angle: 20+
Mean MAE: 0.0018198301159831527 (0.001849572819095476)
focal_length: 20+
Mean MAE: 0.018935315528975125 (0.014115644147768355)
temperature: 2
Mean MAE: 0.00339794414875203 (0.0017462109531530782)
oscillation_frequency: 20+
Mean MAE: 0.002275734942404206 (0.0012322427262526725)
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jul/2023 23:17:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 23:17:08] "GET /_dash-component-suites/dash/deps/react@16.v2_9_3m1684429620.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 23:17:08] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_3m1684429620.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 23:17:08] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_3m1684429620.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 23:17:08] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_3m1684429620.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 23:17:08] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_9_2m1684429619.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 23:17:08] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_9_3m1684429618.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 23:17:08] "GET /_dash-component-s